## Test

In [ ]:
test_sample = 1

In [ ]:
p = lda_model.transform(X_test[test_sample])
print(p)

In [ ]:
t = p.argmax()
print("Topic #{}".format(t))

In [ ]:
print(X_test_document[test_sample])

## Test Ensemble Prediction

In [ ]:
# Get a specific test sample
test_sample = 1

# Transform the test sample using LDA model
p = lda_model.transform(X_test[test_sample])
t = p.argmax()

# Print the topic probabilities and the dominant topic
print("Topic Probabilities:", p)
print("Dominant Topic: #{}".format(t))

# Print the corresponding test document
print("Test Document:", X_test_document[test_sample])

# Make predictions on the test data using the fitted ensemble model
y_pred = grid_search_ensemble.predict(X_test)

# Print the predictions
print("Predictions:", y_pred)
